<a href="https://colab.research.google.com/github/Pariskimhj/AI_class/blob/master/%EC%B1%97%EB%B4%87%EB%A7%8C%EB%93%A4%EA%B8%B03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('treebank')
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer # 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 패딩
from tensorflow.keras.utils import to_categorical # 원핫벡터
from sklearn.model_selection import train_test_split # 학습데이터 분리
tagged_sentences = nltk.corpus.treebank.tagged_sents()
print("품사 태깅이 된 문장 개수: ", len(tagged_sentences))
tagged_sentences[0]
sentences, pos_tags = [], [] 
for tagged_sentence in tagged_sentences: # 3,914개의 문장 샘플을 1개씩 불러온다.
    sentence, tag_info = zip(tagged_sentence) # 각 샘플에서 단어들은 sentence에 품사 태깅 정보들은 tag_info에 저장한다.
    # zip 함수에를 사용하면 각 열끼리 데이터를 묶게 된다. 지금은 3914행 2열이개 때문에 결과가 두 개의 데이터가 나오게 되고 반복문에 의해 입력된다.
    sentences.append(list(sentence)) # 각 샘플에서 단어 정보만 저장한다.
    pos_tags.append(list(tag_info)) # 각 샘플에서 품사 태깅 정보만 저장한다.
[len(tagged_sentences[i]) for i in range(10)]
print('샘플의 최대 길이 : %d' % max(len(l) for l in sentences))
print('샘플의 평균 길이 : %f' % (sum(map(len, sentences))/len(sentences))) # map(함수, 복수데이터)
plt.hist(list(map(len, sentences)), bins=100)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()
src_tokenizer = tokenize(sentences)
tar_tokenizer = tokenize(pos_tags)
vocab_size = len(src_tokenizer.word_index) + 1
tag_size = len(tar_tokenizer.word_index) + 1
print('단어 집합의 크기 : {}'.format(vocab_size))
print('태깅 정보 집합의 크기 : {}'.format(tag_size))
X_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(pos_tags)
max_len = 100
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
y_train = pad_sequences(y_train, padding='post', maxlen=max_len)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.2, random_state=777)
print('훈련 샘플 문장의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
from tensorflow.keras.optimizers import Adam

embedding_dim = 128
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, mask_zero=True)) # 
model.add(Bidirectional(LSTM(hidden_units, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation=('softmax'))))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=7, validation_data=(X_test, y_test))
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))
print('X_test의 길이:',len(X_test[10]))
X_test[10]
print('y_test의 길이:',len(y_test[10]))
print(y_test[10].shape)
y_test[10]
y_predicted = model.predict(np.array([X_test[10]])) # 입력한 테스트용 샘플에 대해서 예측값 y를 리턴
print(len(y_predicted[0]))
print(y_predicted.shape)
y_predicted[0]
y_predicted = np.argmax(y_predicted, axis=-1)
print(len(y_predicted[0]))
print(y_predicted.shape)
y_predicted[0]
def tokenize(samples):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(samples)
  return tokenizer
index_to_word = src_tokenizer.index_word
index_to_tag = tar_tokenizer.index_word

i = 14 # 확인하고 싶은 테스트용 샘플의 인덱스
y_predicted = model.predict(np.array([X_test[i]])) # 입력한 테스트용 샘플에 대해서 예측값 y를 리턴
y_predicted = np.argmax(y_predicted, axis=-1) # 확률 벡터를 정수 레이블로 변환.

print("{:16}{:8}{:8}{}".format("단어", "실제값", "예측값", "결과"))

for word, tag, pred in zip(X_test[i], y_test[i], y_predicted[0]):
    if word != 0: # PAD값은 제외함.
        print("{:16}: {:9}{:8}{}".format(index_to_word[word], 
                                      index_to_tag[tag].upper(), 
                                      index_to_tag[pred].upper(),
                                      tag == pred))